In [1]:
from os import walk
import re
import gzip

In [4]:
trace_path="/home/tiratatp/Repositories/snia_traces"
file_pattern=re.compile("disk(\d+)\.txt(.gz)?")

# cutoff sizes for "small I/O", in KB
sizes = [0, 4, 8, 16, 32, 64, 128, 256, ]

# bin size in ms; 1hour
bin_size = 3600000

In [5]:
def tail(f, window=20):
    """
    Returns the last `window` lines of file `f` as a list.
    """
    if window == 0:
        return []
    BUFSIZ = 1024
    f.seek(0, 2)
    bytes = f.tell()
    size = window + 1
    block = -1
    data = []
    while size > 0 and bytes > 0:
        if bytes - BUFSIZ > 0:
            # Seek back one whole BUFSIZ
            f.seek(block * BUFSIZ, 2)
            # read BUFFER
            data.insert(0, f.read(BUFSIZ))
        else:
            # file too small, start from begining
            f.seek(0,0)
            # only read what was not read
            data.insert(0, f.read(bytes))
        linesFound = data[0].count('\n')
        size -= linesFound
        bytes -= BUFSIZ
        block -= 1
    return ''.join(data).splitlines()[-window:]

In [17]:
traces = {}
sb_all = {}
sb_busy = {}
sio_all = {}
sio_busy = {}
pio_all_w = {}
ps_all_w = {}

for root, dirs, files in walk(trace_path):
    if root[-9:] != '/Combined':
        continue
        
    trace=root.split("/")[-3]
         
    for f in files:
        m = file_pattern.match(f)
        if m is None:
            continue
        is_gzip = True if m.lastindex == 2 else False
        #disk = int(m.group(1))
                    
        #print trace, f, disk
        file_path = "%s/%s" % (root, f)
        
        if is_gzip:
            t = gzip.GzipFile(file_path, "r")
            first = t.next()
            for line in t:
                last = line
            
        else:
            t = open(file_path, "r")
            first = t.next()
            last = tail(t, window=1)
            last = last[0]
            
        
        print trace, f, ((float(last.split(' ')[0]) - float(first.split(' ')[0])) / 3600000), " Hours"


T3 disk0.txt 24.1000187239  Hours
T10 disk3.txt 28.0155221764  Hours
T10 disk4.txt 33.3576864525  Hours
T10 disk0.txt 33.3570668314  Hours
T10 disk6.txt 33.3542484261  Hours
T10 disk2.txt 28.0155425442  Hours
T10 disk5.txt 33.3365034867  Hours
T10 disk1.txt 33.3575478389  Hours
T8 disk2.txt.gz 24.0503343428  Hours
T8 disk16.txt.gz 24.0503210175  Hours
T8 disk11.txt.gz 24.0503217308  Hours
T8 disk10.txt.gz 24.0503211553  Hours
T8 disk8.txt.gz 24.0503166736  Hours
T8 disk4.txt.gz 24.0503332203  Hours
T8 disk12.txt.gz 24.0503175692  Hours
T8 disk3.txt.gz 24.0503343414  Hours
T8 disk5.txt.gz 24.0852981894  Hours
T8 disk14.txt.gz 24.0503199436  Hours
T8 disk13.txt.gz 0.853202518333  Hours
T8 disk6.txt.gz 24.0801899197  Hours
T8 disk1.txt.gz 24.0801724417  Hours
T8 disk0.txt.gz 24.0851170089  Hours
T8 disk7.txt.gz 24.0503180497  Hours
T8 disk9.txt.gz 24.0697852681  Hours
T1 disk0.txt 24.0638117692  Hours
T1 disk1.txt 24.0638632944  Hours
T5 disk3.txt 6.08201666583  Hours
T5 disk4.txt 6.08202

In [7]:
# header    
write_size_columns.insert(0, "trace")
write_size_columns.pop(-1)
write_size_columns.append("write_size_others")    

print "%s %s" % ("sb_all", "-"*100)
print " ".join(write_size_columns)
for t in sorted(sb_all, key=lambda x: int(x[1:])):
    print "%s %s" % (t, " ".join(map(str, sb_all[t])))
print "%s %s" % ("sb_busy", "-"*100)
print " ".join(write_size_columns)
for t in sorted(sb_busy, key=lambda x: int(x[1:])):
    print "%s %s" % (t, " ".join(map(str, sb_busy[t])))
      
# header    
write_count_columns.insert(0, "trace")
write_count_columns.pop(-1)
write_count_columns.append("write_count_others")    

print "%s %s" % ("sio_all", "-"*100)
print " ".join(write_count_columns)
for t in sorted(sio_all, key=lambda x: int(x[1:])):
    print "%s %s" % (t, " ".join(map(str, sio_all[t])))
print "%s %s" % ("sio_busy", "-"*100)
print " ".join(write_count_columns)
for t in sorted(sb_busy, key=lambda x: int(x[1:])):
    print "%s %s" % (t, " ".join(map(str, sio_busy[t])))
    
print "%s %s" % ("ps_all_w", "-"*100)
print " ".join(write_size_columns)
for t in sorted(ps_all_w, key=lambda x: int(x[1:])):
    print "%s %s" % (t, " ".join(map(str, ps_all_w[t])))      
    
print "%s %s" % ("pio_all_w", "-"*100)
print " ".join(write_count_columns)
for t in sorted(pio_all_w, key=lambda x: int(x[1:])):
    print "%s %s" % (t, " ".join(map(str, pio_all_w[t])))   

sb_all ----------------------------------------------------------------------------------------------------
trace trace write_size_le_8KB write_size_le_16KB write_size_le_32KB write_size_le_64KB write_size_le_128KB write_size_le_256KB write_size_le_512KB write_size_others
T1 0.0102514648438 0.002151222229 0.00289367675781 0.000955657958984 0.70332118988 1.44997619629 2.24364112854 17.5566329575
T2 0.0135818862915 0.00760246276855 0.0109134101868 0.00122501373291 0.000965805053711 0.000746574401855 0.00171691894531 0.00236785888672
T3 0.040986366272 0.012431602478 0.0135290527344 0.0048046875 0.0286703491211 0.0575259017944 0.0903401947021 1.51575679779
T4 0.164332962036 0.654659843445 0.139455451965 0.181035804749 0.404600944519 0.115535392761 0.0116324234009 0.183622894287
T5 0.0546031679426 0.00130476270403 0.00491837092808 0.00553512573242 0.0500781195504 0.0301809310913 0.0182433809553 0.0657231467111
T6 0.00065176827567 3.16783251081 0.175985608782 0.0398385184152 0.0651136125837 

In [8]:
print traces

{'T8': {'max_write_count': 4630823, 'disk': 6}, 'T9': {'max_write_count': 933205, 'disk': 0}, 'T6': {'max_write_count': 3039992, 'disk': 5}, 'T7': {'max_write_count': 5663693, 'disk': 0}, 'T4': {'max_write_count': 3830994, 'disk': 8}, 'T5': {'max_write_count': 123961, 'disk': 0}, 'T2': {'max_write_count': 140938, 'disk': 0}, 'T3': {'max_write_count': 398555, 'disk': 0}, 'T1': {'max_write_count': 1452455, 'disk': 1}, 'T10': {'max_write_count': 741091, 'disk': 6}}


In [15]:
print traces

{'T8': {'disk': 6, 'max_count': 12681568}, 'T9': {'disk': 0, 'max_count': 1156315}, 'T6': {'disk': 5, 'max_count': 12734396}, 'T7': {'disk': 0, 'max_count': 10974828}, 'T4': {'disk': 6, 'max_count': 8152268}, 'T5': {'disk': 0, 'max_count': 501382}, 'T2': {'disk': 0, 'max_count': 1520468}, 'T3': {'disk': 0, 'max_count': 416889}, 'T1': {'disk': 1, 'max_count': 14013608}, 'T10': {'disk': 4, 'max_count': 856127}}


In [16]:
# header    
write_size_columns.insert(0, "trace")
write_size_columns.pop(-1)
write_size_columns.append("write_size_others")    

print "%s %s" % ("sb_all", "-"*100)
print " ".join(write_size_columns)
for t in sorted(sb_all, key=lambda x: int(x[1:])):
    print "%s %s" % (t, " ".join(map(str, sb_all[t])))
print "%s %s" % ("sb_busy", "-"*100)
print " ".join(write_size_columns)
for t in sorted(sb_busy, key=lambda x: int(x[1:])):
    print "%s %s" % (t, " ".join(map(str, sb_busy[t])))
      
# header    
write_count_columns.insert(0, "trace")
write_count_columns.pop(-1)
write_count_columns.append("write_count_others")    

print "%s %s" % ("sio_all", "-"*100)
print " ".join(write_count_columns)
for t in sorted(sio_all, key=lambda x: int(x[1:])):
    print "%s %s" % (t, " ".join(map(str, sio_all[t])))
print "%s %s" % ("sio_busy", "-"*100)
print " ".join(write_count_columns)
for t in sorted(sb_busy, key=lambda x: int(x[1:])):
    print "%s %s" % (t, " ".join(map(str, sio_busy[t])))
    
print "%s %s" % ("ps_all_w", "-"*100)
print " ".join(write_size_columns)
for t in sorted(ps_all_w, key=lambda x: int(x[1:])):
    print "%s %s" % (t, " ".join(map(str, ps_all_w[t])))      
    
print "%s %s" % ("pio_all_w", "-"*100)
print " ".join(write_count_columns)
for t in sorted(pio_all_w, key=lambda x: int(x[1:])):
    print "%s %s" % (t, " ".join(map(str, pio_all_w[t])))   

sb_all ----------------------------------------------------------------------------------------------------
trace write_size_le_4KB write_size_le_8KB write_size_le_16KB write_size_le_32KB write_size_le_64KB write_size_le_128KB write_size_le_256KB write_size_others
T1 0.0102514648438 0.002151222229 0.00289367675781 0.000955657958984 0.70332118988 1.44997619629 2.24364112854 17.5566329575
T2 0.0135818862915 0.00760246276855 0.0109134101868 0.00122501373291 0.000965805053711 0.000746574401855 0.00171691894531 0.00236785888672
T3 0.040986366272 0.012431602478 0.0135290527344 0.0048046875 0.0286703491211 0.0575259017944 0.0903401947021 1.51575679779
T4 0.170773468018 0.629287338257 0.133458061218 0.198396034241 0.42244430542 0.0591884231567 0.011199760437 0.186687469482
T5 0.0546031679426 0.00130476270403 0.00491837092808 0.00553512573242 0.0500781195504 0.0301809310913 0.0182433809553 0.0657231467111
T6 0.00065176827567 3.16783251081 0.175985608782 0.0398385184152 0.0651136125837 0.0995112